In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install memory-profiler
!pip install --upgrade psutil
!pip install datasketch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 KB 3.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip
import struct
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import jaccard_score
from sklearn.metrics import classification_report
import scipy
import psutil
from memory_profiler import memory_usage
from sklearn.metrics.pairwise import pairwise_distances
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

In [5]:
'''!wget -P /content/drive/MyDrive/MLBD_Assignment_2/ http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!wget -P /content/drive/MyDrive/MLBD_Assignment_2/ http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!wget -P /content/drive/MyDrive/MLBD_Assignment_2/ http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!wget -P /content/drive/MyDrive/MLBD_Assignment_2/ http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'''

'!wget -P /content/drive/MyDrive/MLBD_Assignment_2/ http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz\n!wget -P /content/drive/MyDrive/MLBD_Assignment_2/ http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz\n!wget -P /content/drive/MyDrive/MLBD_Assignment_2/ http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz\n!wget -P /content/drive/MyDrive/MLBD_Assignment_2/ http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'

In [6]:
def jaccard_distance(x1, x2):
  return 1 - jaccard_score(x1, x2)

def binarizer(images):
    binary_images = np.where(images > 0, 1, 0).astype('uint8')
    return binary_images

def KNN_Classification(n, mode, tr_img, tr_lab, te_img, te_lab, subset):
  if mode==1:
    knn = KNeighborsClassifier(n_neighbors=n, metric=jaccard_distance, n_jobs=1)
  if mode==2:
    knn = KNeighborsClassifier(n_neighbors=n, metric='jaccard', n_jobs=1)
  if mode==3:
    knn = KNeighborsClassifier(n_neighbors=n, metric=scipy.spatial.distance.jaccard, n_jobs=1)
  start_tr = time.time()
  knn.fit(tr_img, tr_lab)
  end_tr = time.time()
  fit_tr = end_tr - start_tr
  print("Time taken to fit KNN model:", fit_tr, "seconds")
  start_te = time.time()
  te_pred = knn.predict(te_img[:subset+1,:])
  print(classification_report(te_lab[:subset+1], te_pred))
  end_te = time.time()
  fit_te = end_te - start_te
  print("Time taken to evaluate KNN model on test portion:", fit_te, "seconds\n")
  del knn

def KNN_Information(n, mode, tr_img, tr_lab, te_img, te_lab, subset):
  if mode==1:
    knn = KNeighborsClassifier(n_neighbors=n, metric=jaccard_distance, n_jobs=1)
  if mode==2:
    knn = KNeighborsClassifier(n_neighbors=n, metric='jaccard', n_jobs=1)
  if mode==3:
    knn = KNeighborsClassifier(n_neighbors=n, metric=scipy.spatial.distance.jaccard, n_jobs=1)
  mem_usage = memory_usage((knn.fit, (tr_img, tr_lab)), max_usage=True, interval=0.1)
  mem_usage_peak = mem_usage
  mem_usage2 = memory_usage((knn.predict, (te_img[:subset+1,:],)), max_usage=True, interval=0.1)
  mem_usage2_peak = mem_usage2
  ncpus = psutil.cpu_count(logical=False)
  l1_cache_size = 32 * 1024 // ncpus
  l2_cache_size = 256 * 1024 // ncpus
  knn_mem_usage = memory_usage((lambda: (knn, knn._fit_X, knn._y)), interval=0.1)
  knn_mem_usage = np.max(knn_mem_usage) - np.min(knn_mem_usage)
  process = psutil.Process()
  resident_mem_usage = process.memory_info().rss
  virtual_mem_usage = process.memory_info().vms
  cpu_usage = process.cpu_percent()
  num_threads = process.num_threads()
  num_fds = process.num_fds()
  print(f"Peak memory usage during fit function: {mem_usage_peak:.2f} MiB")
  print(f"Peak memory usage during predict function: {mem_usage2_peak:.2f} MiB")
  print(f"L1 cache size: {l1_cache_size / 1024:.2f} KiB")
  print(f"L2 cache size: {l2_cache_size / 1024:.2f} KiB")
  print(f"Memory usage of KNN object and its attributes: {knn_mem_usage:.2f} MiB")
  print(f"Resident memory usage of Python process: {resident_mem_usage / 1024 / 1024:.2f} MiB")
  print(f"Virtual memory usage of Python process: {virtual_mem_usage / 1024 / 1024:.2f} MiB")
  print(f"CPU usage of Python process: {cpu_usage:.2f}%")
  print(f"Number of threads of Python process: {num_threads}")
  print(f"Number of open file descriptors of Python process: {num_fds}")


def LSH_KNN(n, mode, tr_img, tr_lab, te_img, te_lab, subset, n_hashes=30, s=0.9):
  n_cols = tr_img.shape[1]
  hash_funcs = [np.random.permutation(n_cols) for _ in range(n_hashes)]
  train_sig = np.full((tr_img.shape[0], n_hashes), np.inf)
  test_sig = np.full((te_img.shape[0], n_hashes), np.inf)
  for i in range(tr_img.shape[0]):
    for j in range(n_hashes):
      idx = np.argmax(tr_img[i, hash_funcs[j]]) 
      train_sig[i, j] = idx
  for i in range(te_img.shape[0]):
    for j in range(n_hashes):
      idx = np.argmax(te_img[i, hash_funcs[j]])
      test_sig[i, j] = idx
  distances = pairwise_distances(train_sig, test_sig, metric='jaccard')
  similarities = 1 - distances
  similarities[similarities < s] = 0
  if mode==1:
    knn = KNeighborsClassifier(n_neighbors=n, metric=jaccard_distance, n_jobs=1)
  if mode==2:
    knn = KNeighborsClassifier(n_neighbors=n, metric='jaccard', n_jobs=1)
  if mode==3:
    knn = KNeighborsClassifier(n_neighbors=n, metric=scipy.spatial.distance.jaccard, n_jobs=1)
  start_tr = time.time()
  knn.fit(train_sig, tr_lab)
  end_tr = time.time()
  fit_tr = end_tr - start_tr
  print("Time taken to fit KNN model:", fit_tr, "seconds")
  start_te = time.time()
  te_pred = knn.predict(test_sig[:subset+1,:])
  print(classification_report(te_lab[:subset+1], te_pred))
  end_te = time.time()
  fit_te = end_te - start_te
  print("Time taken to evaluate KNN model on test portion:", fit_te, "seconds\n")
  del knn

def LSH_Information(n, mode, tr_img, tr_lab, te_img, te_lab, subset, n_hashes=30, s=0.9):
  n_cols = tr_img.shape[1]
  hash_funcs = [np.random.permutation(n_cols) for _ in range(n_hashes)]
  train_sig = np.full((tr_img.shape[0], n_hashes), np.inf)
  test_sig = np.full((te_img.shape[0], n_hashes), np.inf)
  for i in range(tr_img.shape[0]):
    for j in range(n_hashes):
      idx = np.argmax(tr_img[i, hash_funcs[j]]) 
      train_sig[i, j] = idx
  for i in range(te_img.shape[0]):
    for j in range(n_hashes):
      idx = np.argmax(te_img[i, hash_funcs[j]])
      test_sig[i, j] = idx
  distances = pairwise_distances(train_sig, test_sig, metric='jaccard')
  similarities = 1 - distances
  similarities[similarities < s] = 0
  if mode==1:
    knn = KNeighborsClassifier(n_neighbors=n, metric=jaccard_distance, n_jobs=1)
  if mode==2:
    knn = KNeighborsClassifier(n_neighbors=n, metric='jaccard', n_jobs=1)
  if mode==3:
    knn = KNeighborsClassifier(n_neighbors=n, metric=scipy.spatial.distance.jaccard, n_jobs=1)
  mem_usage = memory_usage((knn.fit, (train_sig, tr_lab)), max_usage=True, interval=0.1)
  mem_usage_peak = mem_usage
  mem_usage2 = memory_usage((knn.predict, (test_sig[:subset+1,:],)), max_usage=True, interval=0.1)
  mem_usage2_peak = mem_usage2
  ncpus = psutil.cpu_count(logical=False)
  l1_cache_size = 32 * 1024 // ncpus
  l2_cache_size = 256 * 1024 // ncpus
  knn_mem_usage = memory_usage((lambda: (knn, knn._fit_X, knn._y)), interval=0.1)
  knn_mem_usage = np.max(knn_mem_usage) - np.min(knn_mem_usage)
  process = psutil.Process()
  resident_mem_usage = process.memory_info().rss
  virtual_mem_usage = process.memory_info().vms
  cpu_usage = process.cpu_percent()
  num_threads = process.num_threads()
  num_fds = process.num_fds()
  print(f"Peak memory usage during fit function: {mem_usage_peak:.2f} MiB")
  print(f"Peak memory usage during predict function: {mem_usage2_peak:.2f} MiB")
  print(f"L1 cache size: {l1_cache_size / 1024:.2f} KiB")
  print(f"L2 cache size: {l2_cache_size / 1024:.2f} KiB")
  print(f"Memory usage of KNN object and its attributes: {knn_mem_usage:.2f} MiB")
  print(f"Resident memory usage of Python process: {resident_mem_usage / 1024 / 1024:.2f} MiB")
  print(f"Virtual memory usage of Python process: {virtual_mem_usage / 1024 / 1024:.2f} MiB")
  print(f"CPU usage of Python process: {cpu_usage:.2f}%")
  print(f"Number of threads of Python process: {num_threads}")
  print(f"Number of open file descriptors of Python process: {num_fds}")
  del knn, similarities

In [7]:
with gzip.open('/content/drive/MyDrive/MLBD_Assignment_2/train-images-idx3-ubyte.gz', 'rb') as f:
    _, num_images, num_rows, num_cols = struct.unpack('>IIII', f.read(16))
    images_binary = f.read(num_images*num_rows*num_cols)
    tr_images = np.frombuffer(images_binary, dtype=np.uint8)
    tr_images = tr_images.reshape(num_images, num_rows, num_cols)

with gzip.open('/content/drive/MyDrive/MLBD_Assignment_2/train-labels-idx1-ubyte.gz', 'rb') as f:
    _, num_labels = struct.unpack('>II', f.read(8))
    labels_binary = f.read(num_labels)
    tr_labels = np.frombuffer(labels_binary, dtype=np.uint8)

In [8]:
with gzip.open('/content/drive/MyDrive/MLBD_Assignment_2/t10k-images-idx3-ubyte.gz', 'rb') as f:
    _, num_images, num_rows, num_cols = struct.unpack('>IIII', f.read(16))
    images_binary = f.read(num_images*num_rows*num_cols)
    te_images = np.frombuffer(images_binary, dtype=np.uint8)
    te_images = te_images.reshape(num_images, num_rows, num_cols)

with gzip.open('/content/drive/MyDrive/MLBD_Assignment_2/t10k-labels-idx1-ubyte.gz', 'rb') as f:
    _, num_labels = struct.unpack('>II', f.read(8))
    labels_binary = f.read(num_labels)
    te_labels = np.frombuffer(labels_binary, dtype=np.uint8)

In [9]:
tr_images = tr_images.reshape(tr_images.shape[0],-1)
te_images = te_images.reshape(te_images.shape[0],-1)
print(tr_images.shape, te_images.shape)

(60000, 784) (10000, 784)


In [10]:
tr_images, te_images = binarizer(tr_images), binarizer(te_images)

# With Custom Function

## Without LSH

### Time and Accuracy

In [ ]:
for imgs in [10,60,90]:  
  for k in [1,2,3,4,5]:
    KNN_Classification(k, 1, tr_images, tr_labels, te_images, te_labels, imgs)

### Memory related information for imgs=10

In [ ]:
KNN_Information(1, 1, tr_images, tr_labels, te_images, te_labels, 10)

In [ ]:
KNN_Information(2, 1, tr_images, tr_labels, te_images, te_labels, 10)

In [ ]:
KNN_Information(3, 1, tr_images, tr_labels, te_images, te_labels, 10)

In [ ]:
KNN_Information(4, 1, tr_images, tr_labels, te_images, te_labels, 10)

In [ ]:
KNN_Information(5, 1, tr_images, tr_labels, te_images, te_labels, 10)

### Memory related information for imgs=60

In [ ]:
KNN_Information(1, 1, tr_images, tr_labels, te_images, te_labels, 60)

In [ ]:
KNN_Information(2, 1, tr_images, tr_labels, te_images, te_labels, 60)

In [ ]:
KNN_Information(3, 1, tr_images, tr_labels, te_images, te_labels, 60)

In [ ]:
KNN_Information(4, 1, tr_images, tr_labels, te_images, te_labels, 60)

In [ ]:
KNN_Information(5, 1, tr_images, tr_labels, te_images, te_labels, 60)

### Memory related information for imgs=90

In [ ]:
KNN_Information(1, 1, tr_images, tr_labels, te_images, te_labels, 90)

In [ ]:
KNN_Information(2, 1, tr_images, tr_labels, te_images, te_labels, 90)

In [ ]:
KNN_Information(3, 1, tr_images, tr_labels, te_images, te_labels, 90)

In [ ]:
KNN_Information(4, 1, tr_images, tr_labels, te_images, te_labels, 90)

In [ ]:
KNN_Information(5, 1, tr_images, tr_labels, te_images, te_labels, 90)

# With Jaccard Distance Inbuilt function in sklearn

## Without LSH

### Time and Accuracy

In [ ]:
KNN_Classification(1,2,tr_images,tr_labels,te_images,te_labels,10)
KNN_Classification(2,2,tr_images,tr_labels,te_images,te_labels,10)
KNN_Classification(3,2,tr_images,tr_labels,te_images,te_labels,10)
KNN_Classification(4,2,tr_images,tr_labels,te_images,te_labels,10)
KNN_Classification(5,2,tr_images,tr_labels,te_images,te_labels,10)

Time taken to fit KNN model: 0.010190963745117188 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 0.3793461322784424 seconds

Time taken to fit KNN model: 0.005331516265869141 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 0.3873896598815918 seconds

Time taken to fit KNN model: 0.005692481994628906 seconds


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 0.3811318874359131 seconds

Time taken to fit KNN model: 0.004737138748168945 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 0.3861565589904785 seconds

Time taken to fit KNN model: 0.005033969879150391 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 0.39719343185424805 seconds



In [ ]:
KNN_Classification(1,2,tr_images,tr_labels,te_images,te_labels,60)
KNN_Classification(2,2,tr_images,tr_labels,te_images,te_labels,60)
KNN_Classification(3,2,tr_images,tr_labels,te_images,te_labels,60)
KNN_Classification(4,2,tr_images,tr_labels,te_images,te_labels,60)
KNN_Classification(5,2,tr_images,tr_labels,te_images,te_labels,60)

Time taken to fit KNN model: 0.01949167251586914 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_div

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      0.80      0.89         5
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           8       0.00      0.00      0.00         0
           9       1.00      1.00      1.00         6

    accuracy                           0.98        61
   macro avg       0.90      0.88      0.89        61
weighted avg       1.00      0.98      0.99        61

Time taken to evaluate KNN model on test portion: 2.820917844772339 seconds

Time taken to fit KNN model: 0.0052013397216796875 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      0.80      0.89         5
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         6

    accuracy                           0.98        61
   macro avg       0.98      0.98      0.98        61
weighted avg       0.99      0.98      0.98        61

Time taken to evaluate KNN model on test portion: 2.0198285579681396 seconds

Time taken to fit KNN model: 0.005442380905151367 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         6

    accuracy                           1.00        61
   macro avg       1.00      1.00      1.00        61
weighted avg       1.00      1.00      1.00        61

Time taken to evaluate KNN model on test portion: 2.038645029067993 seconds

Time taken to fit KNN model: 0.005765676498413086 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         6

    accuracy                           1.00        61
   macro avg       1.00      1.00      1.00        61
weighted avg       1.00      1.00      1.00        61

Time taken to evaluate KNN model on test portion: 2.0332701206207275 seconds



In [ ]:
KNN_Classification(1,2,tr_images,tr_labels,te_images,te_labels,90)
KNN_Classification(2,2,tr_images,tr_labels,te_images,te_labels,90)
KNN_Classification(3,2,tr_images,tr_labels,te_images,te_labels,90)
KNN_Classification(4,2,tr_images,tr_labels,te_images,te_labels,90)
KNN_Classification(5,2,tr_images,tr_labels,te_images,te_labels,90)

Time taken to fit KNN model: 0.013292074203491211 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       1.00      0.90      0.95        10
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        14
           8       0.67      1.00      0.80         2
           9       1.00      1.00      1.00         9

    accuracy                           0.99        91
   macro avg       0.97      0.99      0.97        91
weighted avg       0.99      0.99      0.99        91

Time taken to evaluate KNN model on test portion: 4.124956369400024 seconds

Time taken to fit KNN model: 0.0047206878662109375 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       0.89      1.00      0.94         8
           3       1.00      0.90      0.95        10
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        14
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         9

    accuracy                           0.99        91
   macro avg       0.99      0.99      0.99        91
weighted avg       0.99      0.99      0.99        91

Time taken to evaluate KNN model on test portion: 2.987200975418091 seconds

Time taken to fit KNN model: 0.004838228225708008 seconds


...

              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       1.00      0.90      0.95        10
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        14
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         9

    accuracy                           0.99        91
   macro avg       0.99      0.99      0.99        91
weighted avg       0.99      0.99      0.99        91

Time taken to evaluate KNN model on test portion: 2.994518280029297 seconds

Time taken to fit KNN model: 0.005452871322631836 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        14
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         9

    accuracy                           1.00        91
   macro avg       1.00      1.00      1.00        91
weighted avg       1.00      1.00      1.00        91

Time taken to evaluate KNN model on test portion: 3.203073501586914 seconds

Time taken to fit KNN model: 0.004694700241088867 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00        10
           4       1.00      0.92      0.96        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        14
           8       1.00      1.00      1.00         2
           9       0.90      1.00      0.95         9

    accuracy                           0.99        91
   macro avg       0.99      0.99      0.99        91
weighted avg       0.99      0.99      0.99        91

Time taken to evaluate KNN model on test portion: 3.910271167755127 seconds



In [ ]:
KNN_Classification(1,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0])
KNN_Classification(2,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0])
KNN_Classification(3,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0])
KNN_Classification(4,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0])

Time taken to fit KNN model: 0.007369279861450195 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.96      0.99      0.98       980
           1       0.97      0.99      0.98      1135
           2       0.98      0.96      0.97      1032
           3       0.94      0.95      0.95      1010
           4       0.98      0.93      0.95       982
           5       0.98      0.92      0.95       892
           6       0.97      0.99      0.98       958
           7       0.97      0.95      0.96      1028
           8       0.94      0.95      0.95       974
           9       0.91      0.95      0.93      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000

Time taken to evaluate KNN model on test portion: 351.071163892746 seconds

Time taken to fit KNN model: 0.00490260124206543 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.94      1.00      0.96       980
           1       0.96      0.99      0.98      1135
           2       0.97      0.97      0.97      1032
           3       0.92      0.96      0.94      1010
           4       0.97      0.96      0.96       982
           5       0.98      0.92      0.95       892
           6       0.97      0.98      0.98       958
           7       0.96      0.96      0.96      1028
           8       0.97      0.92      0.95       974
           9       0.96      0.93      0.94      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000

Time taken to evaluate KNN model on test portion: 349.31530237197876 seconds

Time taken to fit KNN model: 0.00847482681274414 seconds


...

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       980
           1       0.97      0.99      0.98      1135
           2       0.98      0.97      0.97      1032
           3       0.96      0.95      0.95      1010
           4       0.98      0.94      0.96       982
           5       0.98      0.93      0.95       892
           6       0.97      0.98      0.98       958
           7       0.98      0.95      0.96      1028
           8       0.95      0.96      0.95       974
           9       0.92      0.95      0.94      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000

Time taken to evaluate KNN model on test portion: 352.9322168827057 seconds

Time taken to fit KNN model: 0.005086421966552734 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.95      1.00      0.97       980
           1       0.97      0.99      0.98      1135
           2       0.98      0.97      0.98      1032
           3       0.94      0.96      0.95      1010
           4       0.98      0.95      0.96       982
           5       0.99      0.93      0.95       892
           6       0.97      0.98      0.97       958
           7       0.97      0.96      0.96      1028
           8       0.96      0.95      0.95       974
           9       0.93      0.95      0.94      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000

Time taken to evaluate KNN model on test portion: 353.1395151615143 seconds

Time taken to fit KNN model: 0.005254268646240234 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.95      1.00      0.97       980
           1       0.97      0.99      0.98      1135
           2       0.98      0.97      0.97      1032
           3       0.95      0.96      0.95      1010
           4       0.98      0.94      0.96       982
           5       0.99      0.93      0.96       892
           6       0.97      0.98      0.98       958
           7       0.97      0.95      0.96      1028
           8       0.95      0.96      0.95       974
           9       0.91      0.95      0.93      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000

Time taken to evaluate KNN model on test portion: 357.85477685928345 seconds



### Memory related information for imgs=10

In [41]:
KNN_Information(1, 2, tr_images, tr_labels, te_images, te_labels, 10)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9488.35 MiB
Peak memory usage during predict function: 9533.29 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9488.64 MiB
Virtual memory usage of Python process: 10654.89 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [42]:
KNN_Information(2, 2, tr_images, tr_labels, te_images, te_labels, 10)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9488.65 MiB
Peak memory usage during predict function: 9533.52 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9488.65 MiB
Virtual memory usage of Python process: 10654.89 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [43]:
KNN_Information(3, 2, tr_images, tr_labels, te_images, te_labels, 10)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9488.65 MiB
Peak memory usage during predict function: 9533.52 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9488.65 MiB
Virtual memory usage of Python process: 10654.89 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [44]:
KNN_Information(4, 2, tr_images, tr_labels, te_images, te_labels, 10)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9488.65 MiB
Peak memory usage during predict function: 9533.52 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9488.65 MiB
Virtual memory usage of Python process: 10654.89 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [45]:
KNN_Information(5, 2, tr_images, tr_labels, te_images, te_labels, 10)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9488.65 MiB
Peak memory usage during predict function: 9533.52 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9488.65 MiB
Virtual memory usage of Python process: 10654.89 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


### Memory related information for imgs=60

In [46]:
KNN_Information(1, 2, tr_images, tr_labels, te_images, te_labels, 60)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9488.66 MiB
Peak memory usage during predict function: 9533.54 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.21 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [47]:
KNN_Information(2, 2, tr_images, tr_labels, te_images, te_labels, 60)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.21 MiB
Peak memory usage during predict function: 9533.71 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.21 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [48]:
KNN_Information(3, 2, tr_images, tr_labels, te_images, te_labels, 60)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.21 MiB
Peak memory usage during predict function: 9533.58 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.21 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [49]:
KNN_Information(4, 2, tr_images, tr_labels, te_images, te_labels, 60)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.21 MiB
Peak memory usage during predict function: 9533.71 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.21 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [50]:
KNN_Information(5, 2, tr_images, tr_labels, te_images, te_labels, 60)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.21 MiB
Peak memory usage during predict function: 9533.62 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.21 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


### Memory related information for imgs=90

In [51]:
KNN_Information(1, 2, tr_images, tr_labels, te_images, te_labels, 90)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.21 MiB
Peak memory usage during predict function: 9547.05 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.21 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [52]:
KNN_Information(2, 2, tr_images, tr_labels, te_images, te_labels, 90)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.21 MiB
Peak memory usage during predict function: 9547.11 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.21 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [53]:
KNN_Information(3, 2, tr_images, tr_labels, te_images, te_labels, 90)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.21 MiB
Peak memory usage during predict function: 9547.11 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.21 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [54]:
KNN_Information(4, 2, tr_images, tr_labels, te_images, te_labels, 90)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.21 MiB
Peak memory usage during predict function: 9547.46 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.21 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [55]:
KNN_Information(5, 2, tr_images, tr_labels, te_images, te_labels, 90)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.21 MiB
Peak memory usage during predict function: 9547.14 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.21 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


### Memory related information for all test images

In [56]:
KNN_Information(1, 2, tr_images, tr_labels, te_images, te_labels, te_images.shape[0])

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.21 MiB
Peak memory usage during predict function: 11493.12 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.16 MiB
Virtual memory usage of Python process: 10627.42 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [57]:
KNN_Information(2, 2, tr_images, tr_labels, te_images, te_labels, te_images.shape[0])

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.18 MiB
Peak memory usage during predict function: 11497.30 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.23 MiB
Virtual memory usage of Python process: 10627.67 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [58]:
KNN_Information(3, 2, tr_images, tr_labels, te_images, te_labels, te_images.shape[0])

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.23 MiB
Peak memory usage during predict function: 11505.33 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.27 MiB
Virtual memory usage of Python process: 10627.67 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [59]:
KNN_Information(4, 2, tr_images, tr_labels, te_images, te_labels, te_images.shape[0])

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.27 MiB
Peak memory usage during predict function: 11491.36 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.30 MiB
Virtual memory usage of Python process: 10627.67 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


In [60]:
KNN_Information(5, 2, tr_images, tr_labels, te_images, te_labels, te_images.shape[0])

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.30 MiB
Peak memory usage during predict function: 11491.41 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.34 MiB
Virtual memory usage of Python process: 10627.67 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53


## With LSH

### Time and Accuracy

In [11]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,10,40,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.062479257583618164 seconds
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       1.00      1.00      1.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         0
           4       0.67      1.00      0.80         2
           5       0.00      0.00      0.00         1
           7       0.50      1.00      0.67         1
           8       0.00      0.00      0.00         0
           9       1.00      0.50      0.67         2

    accuracy                           0.64        11
   macro avg       0.46      0.44      0.42        11
weighted avg       0.71      0.64      0.63        11

Time taken to evaluate KNN model on test portion: 0.05884909629821777 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008626461029052734 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.67      1.00      0.80         2
           2       1.00      1.00      1.00         1
           4       0.50      0.50      0.50         2
           5       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         2

    accuracy                           0.55        11
   macro avg       0.45      0.50      0.47        11
weighted avg       0.48      0.55      0.51        11

Time taken to evaluate KNN model on test portion: 0.044309377670288086 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011839628219604492 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         0
           4       0.67      1.00      0.80         2
           5       0.00      0.00      0.00         1
           7       0.50      1.00      0.67         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         2

    accuracy                           0.64        11
   macro avg       0.35      0.44      0.39        11
weighted avg       0.53      0.64      0.57        11

Time taken to evaluate KNN model on test portion: 0.07805228233337402 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008510351181030273 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       0.50      1.00      0.67         1
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         1
           8       0.00      0.00      0.00         0
           9       1.00      0.50      0.67         2

    accuracy                           0.64        11
   macro avg       0.50      0.50      0.48        11
weighted avg       0.68      0.64      0.64        11

Time taken to evaluate KNN model on test portion: 0.04380178451538086 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [12]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,10,40,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009089946746826172 seconds
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       1.00      0.50      0.67         2
           2       1.00      1.00      1.00         1
           3       0.00      0.00      0.00         0
           4       1.00      0.50      0.67         2
           5       0.50      1.00      0.67         1
           6       0.00      0.00      0.00         0
           7       0.50      1.00      0.67         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         2

    accuracy                           0.55        11
   macro avg       0.50      0.45      0.43        11
weighted avg       0.73      0.55      0.58        11

Time taken to evaluate KNN model on test portion: 0.037896156311035156 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008667469024658203 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      0.50      0.67         2
           2       1.00      1.00      1.00         1
           4       0.50      0.50      0.50         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         0
           7       0.50      1.00      0.67         1
           9       1.00      0.50      0.67         2

    accuracy                           0.64        11
   macro avg       0.62      0.56      0.56        11
weighted avg       0.77      0.64      0.67        11

Time taken to evaluate KNN model on test portion: 0.046875953674316406 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008722066879272461 seconds
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       1.00      1.00      1.00         2
           2       0.50      1.00      0.67         1
           4       1.00      1.00      1.00         2
           5       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         1
           9       1.00      0.50      0.67         2

    accuracy                           0.82        11
   macro avg       0.74      0.79      0.73        11
weighted avg       0.80      0.82      0.78        11

Time taken to evaluate KNN model on test portion: 0.04538440704345703 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010109901428222656 seconds
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           0.91        11
   macro avg       0.81      0.86      0.83        11
weighted avg       0.85      0.91      0.87        11

Time taken to evaluate KNN model on test portion: 0.04611659049987793 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [13]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,10,60,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.012378215789794922 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      0.50      0.67         2
           2       1.00      1.00      1.00         1
           4       0.67      1.00      0.80         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         0
           7       0.50      1.00      0.67         1
           9       1.00      0.50      0.67         2

    accuracy                           0.73        11
   macro avg       0.65      0.62      0.60        11
weighted avg       0.80      0.73      0.72        11

Time taken to evaluate KNN model on test portion: 0.09534525871276855 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011748313903808594 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 0.06017589569091797 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008639097213745117 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       0.50      1.00      0.67         1
           4       0.67      1.00      0.80         2
           5       0.00      0.00      0.00         1
           7       0.50      1.00      0.67         1
           9       0.00      0.00      0.00         2

    accuracy                           0.73        11
   macro avg       0.52      0.71      0.59        11
weighted avg       0.58      0.73      0.63        11

Time taken to evaluate KNN model on test portion: 0.05669355392456055 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009063005447387695 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       0.50      1.00      0.67         1
           4       0.50      0.50      0.50         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         0
           7       1.00      1.00      1.00         1
           9       0.00      0.00      0.00         2

    accuracy                           0.64        11
   macro avg       0.50      0.56      0.52        11
weighted avg       0.59      0.64      0.61        11

Time taken to evaluate KNN model on test portion: 0.06508612632751465 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [14]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,10,60,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.01157236099243164 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           7       0.50      1.00      0.67         1
           8       0.00      0.00      0.00         0
           9       1.00      0.50      0.67         2

    accuracy                           0.64        11
   macro avg       0.56      0.56      0.54        11
weighted avg       0.68      0.64      0.64        11

Time taken to evaluate KNN model on test portion: 0.055113792419433594 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009945392608642578 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 0.05476236343383789 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009916543960571289 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       0.00      0.00      0.00         1
           7       0.50      1.00      0.67         1
           8       0.00      0.00      0.00         0
           9       1.00      0.50      0.67         2

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.67        11
weighted avg       0.86      0.82      0.82        11

Time taken to evaluate KNN model on test portion: 0.0588374137878418 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008753299713134766 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 0.06262540817260742 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [11]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,60,40,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.03173708915710449 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      0.80      0.89        10
           2       0.50      0.40      0.44         5
           3       0.50      0.40      0.44         5
           4       0.62      0.50      0.56        10
           5       0.44      0.57      0.50         7
           6       0.50      0.60      0.55         5
           7       0.71      0.71      0.71         7
           8       0.00      0.00      0.00         0
           9       0.29      0.33      0.31         6

    accuracy                           0.61        61
   macro avg       0.56      0.53      0.54        61
weighted avg       0.65      0.61      0.62        61

Time taken to evaluate KNN model on test portion: 0.3213772773742676 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       0.83      0.83      0.83         6
           1       0.83      1.00      0.91        10
           2       1.00      0.60      0.75         5
           3       0.75      0.60      0.67         5
           4       0.67      0.80      0.73        10
           5       0.75      0.43      0.55         7
           6       0.83      1.00      0.91         5
           7       0.60      0.86      0.71         7
           8       0.00      0.00      0.00         0
           9       1.00      0.33      0.50         6

    accuracy                           0.74        61
   macro avg       0.73      0.65      0.65        61
weighted avg       0.79      0.74      0.73        61

Time taken to evaluate KNN model on test portion: 0.20363998413085938 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.007482290267944336 seconds
              precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       0.82      0.90      0.86        10
           2       0.50      0.60      0.55         5
           3       1.00      0.60      0.75         5
           4       0.50      0.50      0.50        10
           5       0.80      0.57      0.67         7
           6       0.57      0.80      0.67         5
           7       0.71      0.71      0.71         7
           9       0.43      0.50      0.46         6

    accuracy                           0.67        61
   macro avg       0.70      0.67      0.67        61
weighted avg       0.70      0.67      0.68        61

Time taken to evaluate KNN model on test portion: 0.19919085502624512

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


 seconds



In [16]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,60,40,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010791301727294922 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.50      0.83      0.62         6
           1       1.00      1.00      1.00        10
           2       1.00      0.60      0.75         5
           3       0.43      0.60      0.50         5
           4       0.50      0.30      0.37        10
           5       0.75      0.43      0.55         7
           6       0.43      0.60      0.50         5
           7       0.83      0.71      0.77         7
           8       0.00      0.00      0.00         0
           9       0.43      0.50      0.46         6

    accuracy                           0.62        61
   macro avg       0.59      0.56      0.55        61
weighted avg       0.67      0.62      0.63        61

Time taken to evaluate KNN model on test portion: 0.328930139541626 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       0.50      0.83      0.62         6
           1       0.91      1.00      0.95        10
           2       0.50      0.60      0.55         5
           3       0.75      0.60      0.67         5
           4       0.89      0.80      0.84        10
           5       0.60      0.43      0.50         7
           6       0.50      0.40      0.44         5
           7       0.75      0.86      0.80         7
           8       0.00      0.00      0.00         0
           9       0.67      0.33      0.44         6

    accuracy                           0.69        61
   macro avg       0.61      0.59      0.58        61
weighted avg       0.71      0.69      0.68        61

Time taken to evaluate KNN model on test portion: 0.2171328067779541 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011789083480834961 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.83      0.83      0.83         6
           1       1.00      0.90      0.95        10
           2       0.67      0.80      0.73         5
           3       0.67      0.80      0.73         5
           4       0.89      0.80      0.84        10
           5       0.88      1.00      0.93         7
           6       1.00      0.80      0.89         5
           7       0.67      0.57      0.62         7
           9       0.71      0.83      0.77         6

    accuracy                           0.82        61
   macro avg       0.81      0.82      0.81        61
weighted avg       0.83      0.82      0.82        61

Time taken to evaluate KNN model on test portion: 0.3822958469390869 seconds



In [17]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,60,60,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011161565780639648 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.75      0.50      0.60         6
           1       1.00      1.00      1.00        10
           2       1.00      0.80      0.89         5
           3       0.57      0.80      0.67         5
           4       0.60      0.60      0.60        10
           5       0.80      0.57      0.67         7
           6       0.80      0.80      0.80         5
           7       0.88      1.00      0.93         7
           8       0.00      0.00      0.00         0
           9       0.50      0.50      0.50         6

    accuracy                           0.74        61
   macro avg       0.69      0.66      0.67        61
weighted avg       0.77      0.74      0.75        61

Time taken to evaluate KNN model on test portion: 0.2576746940612793 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.91      1.00      0.95        10
           2       1.00      0.80      0.89         5
           3       0.67      0.80      0.73         5
           4       0.90      0.90      0.90        10
           5       1.00      0.71      0.83         7
           6       0.71      1.00      0.83         5
           7       1.00      0.71      0.83         7
           9       0.86      1.00      0.92         6

    accuracy                           0.89        61
   macro avg       0.89      0.88      0.88        61
weighted avg       0.90      0.89      0.88        61

Time taken to evaluate KNN model on test portion: 0.4830954074859619 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010962247848510742 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      0.50      0.67         6
           1       0.77      1.00      0.87        10
           2       0.67      0.80      0.73         5
           3       0.43      0.60      0.50         5
           4       0.64      0.70      0.67        10
           5       1.00      0.29      0.44         7
           6       1.00      0.60      0.75         5
           7       0.70      1.00      0.82         7
           8       0.00      0.00      0.00         0
           9       0.50      0.33      0.40         6

    accuracy                           0.67        61
   macro avg       0.67      0.58      0.58        61
weighted avg       0.74      0.67      0.66        61

Time taken to evaluate KNN model on test portion: 0.28705596923828125 seconds



In [18]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,60,60,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011441707611083984 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.77      1.00      0.87        10
           2       0.75      0.60      0.67         5
           3       0.75      0.60      0.67         5
           4       0.71      0.50      0.59        10
           5       1.00      0.86      0.92         7
           6       0.75      0.60      0.67         5
           7       1.00      1.00      1.00         7
           9       0.60      1.00      0.75         6

    accuracy                           0.80        61
   macro avg       0.81      0.80      0.79        61
weighted avg       0.81      0.80      0.80        61

Time taken to evaluate KNN model on test portion: 0.24327516555786133 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       0.80      0.80      0.80         5
           3       0.50      0.60      0.55         5
           4       0.89      0.80      0.84        10
           5       0.83      0.71      0.77         7
           6       0.83      1.00      0.91         5
           7       1.00      1.00      1.00         7
           9       0.83      0.83      0.83         6

    accuracy                           0.87        61
   macro avg       0.85      0.86      0.86        61
weighted avg       0.88      0.87      0.87        61

Time taken to evaluate KNN model on test portion: 0.4610023498535156 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010412216186523438 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       0.90      0.90      0.90        10
           2       1.00      0.60      0.75         5
           3       1.00      0.60      0.75         5
           4       0.67      0.80      0.73        10
           5       0.83      0.71      0.77         7
           6       0.83      1.00      0.91         5
           7       0.75      0.86      0.80         7
           8       0.00      0.00      0.00         0
           9       0.60      0.50      0.55         6

    accuracy                           0.79        61
   macro avg       0.74      0.70      0.71        61
weighted avg       0.81      0.79      0.79        61

Time taken to evaluate KNN model on test portion: 0.2788262367248535 seconds



In [19]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,90,40,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.00838780403137207 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.57      0.50      0.53         8
           1       0.64      0.58      0.61        12
           2       0.50      0.38      0.43         8
           3       0.71      0.50      0.59        10
           4       0.76      1.00      0.87        13
           5       0.60      0.43      0.50         7
           6       0.46      0.75      0.57         8
           7       0.62      0.57      0.59        14
           8       0.50      0.50      0.50         2
           9       0.30      0.33      0.32         9

    accuracy                           0.58        91
   macro avg       0.57      0.55      0.55        91
weighted avg       0.59      0.58      0.58        91

Time taken to evaluate KNN model on test portion: 0.2519650459289551 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       1.00      1.00      1.00        12
           2       0.67      0.75      0.71         8
           3       0.83      1.00      0.91        10
           4       0.69      0.69      0.69        13
           5       1.00      0.71      0.83         7
           6       0.86      0.75      0.80         8
           7       0.75      0.64      0.69        14
           8       0.00      0.00      0.00         2
           9       0.30      0.33      0.32         9

    accuracy                           0.75        91
   macro avg       0.69      0.69      0.68        91
weighted avg       0.75      0.75      0.74        91

Time taken to evaluate KNN model on test portion: 0.2952561378479004 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.00848078727722168 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       1.00      1.00      1.00        12
           2       0.71      0.62      0.67         8
           3       1.00      0.60      0.75        10
           4       0.69      0.85      0.76        13
           5       0.71      0.71      0.71         7
           6       0.70      0.88      0.78         8
           7       0.69      0.64      0.67        14
           8       0.33      0.50      0.40         2
           9       0.57      0.44      0.50         9

    accuracy                           0.75        91
   macro avg       0.72      0.72      0.71        91
weighted avg       0.76      0.75      0.74        91

Time taken to evaluate KNN model on test portion: 0.31917905807495117 seconds



In [20]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,90,40,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.0072019100189208984 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.80      0.50      0.62         8
           1       0.90      0.75      0.82        12
           2       0.40      0.50      0.44         8
           3       0.44      0.40      0.42        10
           4       0.57      0.62      0.59        13
           5       0.50      0.71      0.59         7
           6       0.55      0.75      0.63         8
           7       0.64      0.64      0.64        14
           8       0.33      0.50      0.40         2
           9       0.40      0.22      0.29         9

    accuracy                           0.57        91
   macro avg       0.55      0.56      0.54        91
weighted avg       0.59      0.57      0.57        91

Time taken to evaluate KNN model on test portion: 0.24843692779541016 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         8
           1       0.85      0.92      0.88        12
           2       0.60      0.38      0.46         8
           3       0.58      0.70      0.64        10
           4       0.69      0.69      0.69        13
           5       0.40      0.29      0.33         7
           6       0.71      0.62      0.67         8
           7       0.69      0.64      0.67        14
           8       0.00      0.00      0.00         2
           9       0.44      0.44      0.44         9

    accuracy                           0.64        91
   macro avg       0.56      0.57      0.56        91
weighted avg       0.63      0.64      0.63        91

Time taken to evaluate KNN model on test portion: 0.3306267261505127 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.0073087215423583984 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       0.92      1.00      0.96        12
           2       0.75      0.38      0.50         8
           3       0.82      0.90      0.86        10
           4       0.90      0.69      0.78        13
           5       0.71      0.71      0.71         7
           6       0.86      0.75      0.80         8
           7       0.80      0.86      0.83        14
           8       1.00      0.50      0.67         2
           9       0.62      0.89      0.73         9

    accuracy                           0.80        91
   macro avg       0.82      0.77      0.77        91
weighted avg       0.81      0.80      0.79        91

Time taken to evaluate KNN model on test portion: 0.30472350120544434 seconds



In [21]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,90,60,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.00876927375793457 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       1.00      0.83      0.91        12
           2       1.00      0.88      0.93         8
           3       0.67      0.60      0.63        10
           4       0.80      0.92      0.86        13
           5       0.60      0.43      0.50         7
           6       1.00      1.00      1.00         8
           7       0.67      0.86      0.75        14
           8       0.67      1.00      0.80         2
           9       0.67      0.67      0.67         9

    accuracy                           0.80        91
   macro avg       0.81      0.81      0.80        91
weighted avg       0.81      0.80      0.80        91

Time taken to evaluate KNN model on test portion: 0.35521793365478516 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.86      1.00      0.92        12
           2       0.62      0.62      0.62         8
           3       0.88      0.70      0.78        10
           4       0.65      0.85      0.73        13
           5       1.00      0.71      0.83         7
           6       0.78      0.88      0.82         8
           7       0.71      0.86      0.77        14
           8       1.00      0.50      0.67         2
           9       1.00      0.33      0.50         9

    accuracy                           0.78        91
   macro avg       0.84      0.75      0.76        91
weighted avg       0.81      0.78      0.77        91

Time taken to evaluate KNN model on test portion: 0.4062316417694092 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010830163955688477 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.86      1.00      0.92        12
           2       0.62      0.62      0.62         8
           3       1.00      0.80      0.89        10
           4       1.00      1.00      1.00        13
           5       0.88      1.00      0.93         7
           6       1.00      1.00      1.00         8
           7       0.87      0.93      0.90        14
           8       0.50      0.50      0.50         2
           9       0.86      0.67      0.75         9

    accuracy                           0.89        91
   macro avg       0.86      0.85      0.85        91
weighted avg       0.89      0.89      0.89        91

Time taken to evaluate KNN model on test portion: 0.42535972595214844 seconds



In [22]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,90,60,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011450052261352539 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.86      1.00      0.92        12
           2       0.75      0.75      0.75         8
           3       0.67      0.40      0.50        10
           4       0.71      0.77      0.74        13
           5       0.57      0.57      0.57         7
           6       0.88      0.88      0.88         8
           7       0.73      0.57      0.64        14
           8       0.33      0.50      0.40         2
           9       0.18      0.22      0.20         9

    accuracy                           0.68        91
   macro avg       0.66      0.67      0.65        91
weighted avg       0.69      0.68      0.68        91

Time taken to evaluate KNN model on test portion: 0.3572263717651367 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.85      0.92      0.88        12
           2       0.80      1.00      0.89         8
           3       0.71      0.50      0.59        10
           4       0.73      0.85      0.79        13
           5       0.80      0.57      0.67         7
           6       1.00      1.00      1.00         8
           7       0.81      0.93      0.87        14
           8       1.00      1.00      1.00         2
           9       0.50      0.33      0.40         9

    accuracy                           0.80        91
   macro avg       0.81      0.81      0.80        91
weighted avg       0.79      0.80      0.79        91

Time taken to evaluate KNN model on test portion: 0.4330899715423584 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.015122175216674805 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       1.00      1.00      1.00        12
           2       0.86      0.75      0.80         8
           3       0.88      0.70      0.78        10
           4       0.71      0.77      0.74        13
           5       0.71      0.71      0.71         7
           6       0.86      0.75      0.80         8
           7       0.93      0.93      0.93        14
           8       0.33      0.50      0.40         2
           9       0.67      0.67      0.67         9

    accuracy                           0.81        91
   macro avg       0.77      0.78      0.77        91
weighted avg       0.82      0.81      0.81        91

Time taken to evaluate KNN model on test portion: 0.704859733581543 seconds



In [23]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],40,0.8)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],40,0.8)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],40,0.8)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],40,0.8)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],40,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010589838027954102 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.70      0.66      0.68       980
           1       0.91      0.86      0.88      1135
           2       0.76      0.78      0.77      1032
           3       0.65      0.68      0.66      1010
           4       0.58      0.57      0.57       982
           5       0.57      0.54      0.55       892
           6       0.83      0.84      0.83       958
           7       0.56      0.56      0.56      1028
           8       0.53      0.51      0.52       974
           9       0.48      0.54      0.51      1009

    accuracy                           0.66     10000
   macro avg       0.66      0.65      0.65     10000
weighted avg       0.66      0.66      0.66     10000

Time taken to evaluate KNN model on test portion: 37.00922632217407 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       0.81      0.89      0.85       980
           1       0.83      0.94      0.88      1135
           2       0.72      0.71      0.71      1032
           3       0.71      0.73      0.72      1010
           4       0.65      0.77      0.70       982
           5       0.70      0.66      0.68       892
           6       0.84      0.85      0.84       958
           7       0.84      0.80      0.82      1028
           8       0.72      0.58      0.64       974
           9       0.71      0.60      0.65      1009

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000

Time taken to evaluate KNN model on test portion: 37.084452390670776 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009636402130126953 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.77      0.82      0.79       980
           1       0.84      0.96      0.89      1135
           2       0.86      0.75      0.80      1032
           3       0.74      0.74      0.74      1010
           4       0.75      0.80      0.78       982
           5       0.73      0.68      0.70       892
           6       0.84      0.89      0.86       958
           7       0.76      0.81      0.78      1028
           8       0.69      0.59      0.64       974
           9       0.71      0.66      0.68      1009

    accuracy                           0.77     10000
   macro avg       0.77      0.77      0.77     10000
weighted avg       0.77      0.77      0.77     10000

Time taken to evaluate KNN model on test portion: 35.70792269706726 seconds



In [24]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],40,0.9)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],40,0.9)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],40,0.9)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],40,0.9)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],40,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.0073375701904296875 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.80      0.83      0.81       980
           1       0.89      0.92      0.90      1135
           2       0.82      0.74      0.78      1032
           3       0.60      0.59      0.59      1010
           4       0.72      0.68      0.70       982
           5       0.60      0.62      0.61       892
           6       0.69      0.74      0.71       958
           7       0.72      0.72      0.72      1028
           8       0.67      0.59      0.62       974
           9       0.63      0.69      0.66      1009

    accuracy                           0.72     10000
   macro avg       0.71      0.71      0.71     10000
weighted avg       0.72      0.72      0.71     10000

Time taken to evaluate KNN model on test portion: 32.74166202545166 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       0.77      0.93      0.84       980
           1       0.77      0.93      0.84      1135
           2       0.69      0.67      0.68      1032
           3       0.53      0.63      0.58      1010
           4       0.73      0.76      0.74       982
           5       0.66      0.56      0.60       892
           6       0.74      0.63      0.68       958
           7       0.61      0.49      0.55      1028
           8       0.53      0.44      0.48       974
           9       0.59      0.58      0.58      1009

    accuracy                           0.67     10000
   macro avg       0.66      0.66      0.66     10000
weighted avg       0.66      0.67      0.66     10000

Time taken to evaluate KNN model on test portion: 36.70224642753601 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010201692581176758 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.82      0.89      0.86       980
           1       0.85      0.92      0.89      1135
           2       0.79      0.79      0.79      1032
           3       0.71      0.67      0.69      1010
           4       0.58      0.64      0.61       982
           5       0.67      0.65      0.66       892
           6       0.86      0.82      0.84       958
           7       0.72      0.75      0.73      1028
           8       0.74      0.64      0.68       974
           9       0.56      0.53      0.55      1009

    accuracy                           0.73     10000
   macro avg       0.73      0.73      0.73     10000
weighted avg       0.73      0.73      0.73     10000

Time taken to evaluate KNN model on test portion: 34.7724084854126 seconds



In [25]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],60,0.8)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],60,0.8)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],60,0.8)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],60,0.8)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],60,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010690689086914062 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.91      0.94      0.92       980
           1       0.92      0.95      0.93      1135
           2       0.89      0.82      0.85      1032
           3       0.74      0.74      0.74      1010
           4       0.77      0.73      0.75       982
           5       0.78      0.80      0.79       892
           6       0.87      0.89      0.88       958
           7       0.80      0.85      0.82      1028
           8       0.74      0.68      0.71       974
           9       0.70      0.74      0.72      1009

    accuracy                           0.82     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.82      0.81     10000

Time taken to evaluate KNN model on test portion: 43.23662257194519 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       0.88      0.96      0.92       980
           1       0.86      0.98      0.91      1135
           2       0.88      0.82      0.85      1032
           3       0.79      0.81      0.80      1010
           4       0.69      0.78      0.73       982
           5       0.81      0.81      0.81       892
           6       0.90      0.90      0.90       958
           7       0.84      0.83      0.83      1028
           8       0.85      0.69      0.76       974
           9       0.75      0.66      0.70      1009

    accuracy                           0.83     10000
   macro avg       0.83      0.82      0.82     10000
weighted avg       0.83      0.83      0.82     10000

Time taken to evaluate KNN model on test portion: 49.462000608444214 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011110067367553711 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.84      0.90      0.87       980
           1       0.88      0.96      0.92      1135
           2       0.84      0.79      0.81      1032
           3       0.72      0.74      0.73      1010
           4       0.77      0.77      0.77       982
           5       0.70      0.62      0.66       892
           6       0.85      0.88      0.86       958
           7       0.85      0.85      0.85      1028
           8       0.76      0.68      0.72       974
           9       0.73      0.75      0.74      1009

    accuracy                           0.80     10000
   macro avg       0.79      0.80      0.79     10000
weighted avg       0.80      0.80      0.80     10000

Time taken to evaluate KNN model on test portion: 50.58073019981384 seconds



In [26]:
LSH_KNN(1,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],60,0.9)
LSH_KNN(2,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],60,0.9)
LSH_KNN(3,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],60,0.9)
LSH_KNN(4,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],60,0.9)
LSH_KNN(5,2,tr_images,tr_labels,te_images,te_labels,te_images.shape[0],60,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.012070417404174805 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.89      0.90      0.89       980
           1       0.87      0.88      0.87      1135
           2       0.74      0.73      0.73      1032
           3       0.63      0.68      0.65      1010
           4       0.70      0.71      0.71       982
           5       0.70      0.73      0.72       892
           6       0.87      0.87      0.87       958
           7       0.70      0.65      0.67      1028
           8       0.64      0.58      0.61       974
           9       0.60      0.63      0.61      1009

    accuracy                           0.74     10000
   macro avg       0.73      0.73      0.73     10000
weighted avg       0.74      0.74      0.74     10000

Time taken to evaluate KNN model on test portion: 43.046393156051636 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


...

              precision    recall  f1-score   support

           0       0.83      0.95      0.89       980
           1       0.85      0.97      0.91      1135
           2       0.79      0.78      0.79      1032
           3       0.73      0.75      0.74      1010
           4       0.78      0.78      0.78       982
           5       0.84      0.77      0.80       892
           6       0.86      0.89      0.88       958
           7       0.74      0.79      0.76      1028
           8       0.73      0.56      0.63       974
           9       0.71      0.62      0.66      1009

    accuracy                           0.79     10000
   macro avg       0.79      0.79      0.78     10000
weighted avg       0.79      0.79      0.78     10000

Time taken to evaluate KNN model on test portion: 48.07419490814209 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010506629943847656 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


              precision    recall  f1-score   support

           0       0.85      0.93      0.89       980
           1       0.89      0.98      0.93      1135
           2       0.88      0.81      0.84      1032
           3       0.76      0.79      0.78      1010
           4       0.84      0.82      0.83       982
           5       0.78      0.76      0.77       892
           6       0.89      0.90      0.89       958
           7       0.81      0.83      0.82      1028
           8       0.79      0.68      0.74       974
           9       0.72      0.71      0.72      1009

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000

Time taken to evaluate KNN model on test portion: 49.44631862640381 seconds



### Memory related information for imgs=10

In [10]:
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9439.11 MiB
Peak memory usage during predict function: 9449.36 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.37 MiB
Virtual memory usage of Python process: 10594.41 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9439.29 MiB
Peak memory usage during predict function: 9449.44 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.44 MiB
Virtual memory usage of Python process: 10594.40 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9451.37 MiB
Peak memory usage during predict function: 9461.33 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.35 MiB
Virtual memory usage of Python process: 10606.27 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9451.40 MiB
Peak memory usage during predict function: 9461.36 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.36 MiB
Virtual memory usage of Python process: 10606.27 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [11]:
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9439.36 MiB
Peak memory usage during predict function: 9449.54 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.54 MiB
Virtual memory usage of Python process: 10594.40 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9449.56 MiB
Peak memory usage during predict function: 9449.58 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.58 MiB
Virtual memory usage of Python process: 10594.40 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9451.50 MiB
Peak memory usage during predict function: 9461.46 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.48 MiB
Virtual memory usage of Python process: 10606.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.49 MiB
Peak memory usage during predict function: 9461.49 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.49 MiB
Virtual memory usage of Python process: 10606.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [12]:
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.51 MiB
Peak memory usage during predict function: 9461.51 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.51 MiB
Virtual memory usage of Python process: 10606.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.51 MiB
Peak memory usage during predict function: 9461.55 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9461.55 MiB
Virtual memory usage of Python process: 10606.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9461.55 MiB
Peak memory usage during predict function: 9462.05 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.05 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.07 MiB
Peak memory usage during predict function: 9462.08 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.08 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [13]:
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.08 MiB
Peak memory usage during predict function: 9462.11 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.11 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.11 MiB
Peak memory usage during predict function: 9462.12 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.12 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.14 MiB
Peak memory usage during predict function: 9462.15 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.15 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.16 MiB
Peak memory usage during predict function: 9462.17 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.17 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [14]:
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.17 MiB
Peak memory usage during predict function: 9462.18 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.18 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.21 MiB
Peak memory usage during predict function: 9462.21 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.21 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.22 MiB
Peak memory usage during predict function: 9462.24 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.24 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.38 MiB
Peak memory usage during predict function: 9462.38 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.39 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


### Memory related information for imgs=60

In [15]:
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.41 MiB
Peak memory usage during predict function: 9492.45 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9462.41 MiB
Virtual memory usage of Python process: 10606.98 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9462.43 MiB
Peak memory usage during predict function: 9491.85 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9448.59 MiB
Virtual memory usage of Python process: 10593.02 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.48 MiB
Peak memory usage during predict function: 9477.80 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.06 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.49 MiB
Peak memory usage during predict function: 9475.87 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.07 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [16]:
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9449.08 MiB
Peak memory usage during predict function: 9503.86 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9448.62 MiB
Virtual memory usage of Python process: 10593.02 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9448.63 MiB
Peak memory usage during predict function: 9477.86 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9448.63 MiB
Virtual memory usage of Python process: 10593.02 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.51 MiB
Peak memory usage during predict function: 9475.95 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.10 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.60 MiB
Peak memory usage during predict function: 9477.93 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.19 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [17]:
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9449.19 MiB
Peak memory usage during predict function: 9504.06 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.01 MiB
Resident memory usage of Python process: 9448.74 MiB
Virtual memory usage of Python process: 10593.02 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9448.76 MiB
Peak memory usage during predict function: 9483.94 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9448.76 MiB
Virtual memory usage of Python process: 10593.02 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.65 MiB
Peak memory usage during predict function: 9478.03 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.23 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.67 MiB
Peak memory usage during predict function: 9484.04 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.26 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [18]:
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9449.27 MiB
Peak memory usage during predict function: 9500.18 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9448.81 MiB
Virtual memory usage of Python process: 10593.02 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9448.83 MiB
Peak memory usage during predict function: 9468.18 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9448.83 MiB
Virtual memory usage of Python process: 10593.02 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.71 MiB
Peak memory usage during predict function: 9477.99 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.29 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.71 MiB
Peak memory usage during predict function: 9478.21 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.33 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [19]:
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9449.34 MiB
Peak memory usage during predict function: 9504.39 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9448.88 MiB
Virtual memory usage of Python process: 10593.02 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9448.88 MiB
Peak memory usage during predict function: 9504.42 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9448.90 MiB
Virtual memory usage of Python process: 10593.02 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.78 MiB
Peak memory usage during predict function: 9478.27 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.37 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.07 MiB
Peak memory usage during predict function: 9478.38 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.66 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


### Memory related information for imgs=90

In [20]:
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9449.68 MiB
Peak memory usage during predict function: 9490.18 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.70 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9449.70 MiB
Peak memory usage during predict function: 9487.62 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9449.71 MiB
Virtual memory usage of Python process: 10593.48 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.15 MiB
Peak memory usage during predict function: 9490.95 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.15 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.15 MiB
Peak memory usage during predict function: 9489.08 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.16 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [21]:
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.18 MiB
Peak memory usage during predict function: 9493.04 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.19 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.19 MiB
Peak memory usage during predict function: 9491.11 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.20 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.22 MiB
Peak memory usage during predict function: 9514.33 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.23 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.24 MiB
Peak memory usage during predict function: 9491.07 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.24 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [22]:
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.25 MiB
Peak memory usage during predict function: 9491.15 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.28 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.29 MiB
Peak memory usage during predict function: 9536.45 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.29 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.30 MiB
Peak memory usage during predict function: 9493.19 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.30 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.32 MiB
Peak memory usage during predict function: 9530.51 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.33 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [23]:
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.34 MiB
Peak memory usage during predict function: 9493.26 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.34 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.36 MiB
Peak memory usage during predict function: 9495.02 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.36 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.36 MiB
Peak memory usage during predict function: 9516.43 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.01 MiB
Resident memory usage of Python process: 9453.37 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.39 MiB
Peak memory usage during predict function: 9520.49 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.39 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [24]:
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.41 MiB
Peak memory usage during predict function: 9493.27 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.41 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.44 MiB
Peak memory usage during predict function: 9491.26 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.44 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.45 MiB
Peak memory usage during predict function: 9502.50 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.45 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9453.45 MiB
Peak memory usage during predict function: 9493.25 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9453.48 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 52
 


### Memory related information for all test images

In [25]:
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],40,0.8)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],40,0.9)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],60,0.8)
print(" ")
LSH_Information(1,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9453.48 MiB
Peak memory usage during predict function: 11470.10 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9425.49 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9425.95 MiB
Peak memory usage during predict function: 11472.91 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9425.96 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9425.97 MiB
Peak memory usage during predict function: 11460.92 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9425.98 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.01 MiB
Peak memory usage during predict function: 11468.96 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.02 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [26]:
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],40,0.8)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],40,0.9)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],60,0.8)
print(" ")
LSH_Information(2,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.05 MiB
Peak memory usage during predict function: 11469.00 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.07 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.09 MiB
Peak memory usage during predict function: 11425.04 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.10 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.10 MiB
Peak memory usage during predict function: 11431.05 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.11 MiB
Virtual memory usage of Python process: 10597.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 54
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.14 MiB
Peak memory usage during predict function: 11471.09 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.15 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [27]:
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],40,0.8)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],40,0.9)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],60,0.8)
print(" ")
LSH_Information(3,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.15 MiB
Peak memory usage during predict function: 11437.11 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.17 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.20 MiB
Peak memory usage during predict function: 11455.15 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.20 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.21 MiB
Peak memory usage during predict function: 11469.16 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.22 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.26 MiB
Peak memory usage during predict function: 11467.20 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.27 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [28]:
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],40,0.8)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],40,0.9)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],60,0.8)
print(" ")
LSH_Information(4,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.56 MiB
Peak memory usage during predict function: 11471.53 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.58 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.58 MiB
Peak memory usage during predict function: 11471.53 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.59 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.59 MiB
Peak memory usage during predict function: 11467.58 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.64 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.64 MiB
Peak memory usage during predict function: 11457.59 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.65 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [29]:
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],40,0.8)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],40,0.9)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],60,0.8)
print(" ")
LSH_Information(5,2,tr_images,tr_labels,te_images,te_labels, te_images.shape[0],60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.68 MiB
Peak memory usage during predict function: 11473.70 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.68 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.68 MiB
Peak memory usage during predict function: 11473.63 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.01 MiB
Resident memory usage of Python process: 9426.70 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.73 MiB
Peak memory usage during predict function: 11473.69 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.75 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/

Peak memory usage during fit function: 9426.75 MiB
Peak memory usage during predict function: 11473.73 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9426.79 MiB
Virtual memory usage of Python process: 10597.53 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


# With Scipy Jaccard-Needham Dissimilarity

## Without LSH

### Time and Accuracy

In [ ]:
KNN_Classification(1,3,tr_images,tr_labels,te_images,te_labels,10)
KNN_Classification(2,3,tr_images,tr_labels,te_images,te_labels,10)
KNN_Classification(3,3,tr_images,tr_labels,te_images,te_labels,10)
KNN_Classification(4,3,tr_images,tr_labels,te_images,te_labels,10)
KNN_Classification(5,3,tr_images,tr_labels,te_images,te_labels,10)

Time taken to fit KNN model: 0.009647130966186523 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 14.841084957122803 seconds

Time taken to fit KNN model: 0.004952192306518555 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00   

In [ ]:
KNN_Classification(1,3,tr_images,tr_labels,te_images,te_labels,60)
KNN_Classification(2,3,tr_images,tr_labels,te_images,te_labels,60)
KNN_Classification(3,3,tr_images,tr_labels,te_images,te_labels,60)
KNN_Classification(4,3,tr_images,tr_labels,te_images,te_labels,60)
KNN_Classification(5,3,tr_images,tr_labels,te_images,te_labels,60)

Time taken to fit KNN model: 0.006331443786621094 seconds


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      0.80      0.89         5
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           8       0.00      0.00      0.00         0
           9       1.00      1.00      1.00         6

    accuracy                           0.98        61
   macro avg       0.90      0.88      0.89        61
weighted avg       1.00      0.98      0.99        61

Time taken to evaluate KNN model on test portion: 76.35217952728271 seconds

Time taken to fit KNN model: 0.012397289276123047 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        

In [ ]:
KNN_Classification(1,3,tr_images,tr_labels,te_images,te_labels,90)
KNN_Classification(2,3,tr_images,tr_labels,te_images,te_labels,90)
KNN_Classification(3,3,tr_images,tr_labels,te_images,te_labels,90)
KNN_Classification(4,3,tr_images,tr_labels,te_images,te_labels,90)
KNN_Classification(5,3,tr_images,tr_labels,te_images,te_labels,90)

Time taken to fit KNN model: 0.005529880523681641 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       1.00      0.90      0.95        10
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        14
           8       0.67      1.00      0.80         2
           9       1.00      1.00      1.00         9

    accuracy                           0.99        91
   macro avg       0.97      0.99      0.97        91
weighted avg       0.99      0.99      0.99        91

Time taken to evaluate KNN model on test portion: 103.35024881362915 seconds

Time taken to fit KNN model: 0.008364439010620117 seconds
              precision    recall  f1-score   s

### Memory related information for imgs=10

In [ ]:
KNN_Information(1, 3, tr_images, tr_labels, te_images, te_labels, 10)

In [ ]:
KNN_Information(2, 3, tr_images, tr_labels, te_images, te_labels, 10)

In [ ]:
KNN_Information(3, 3, tr_images, tr_labels, te_images, te_labels, 10)

In [ ]:
KNN_Information(4, 3, tr_images, tr_labels, te_images, te_labels, 10)

In [ ]:
KNN_Information(5, 3, tr_images, tr_labels, te_images, te_labels, 10)

### Memory related information for imgs=60

In [ ]:
KNN_Information(1, 3, tr_images, tr_labels, te_images, te_labels, 60)

In [ ]:
KNN_Information(2, 3, tr_images, tr_labels, te_images, te_labels, 60)

In [ ]:
KNN_Information(3, 3, tr_images, tr_labels, te_images, te_labels, 60)

In [ ]:
KNN_Information(4, 3, tr_images, tr_labels, te_images, te_labels, 60)

In [ ]:
KNN_Information(5, 3, tr_images, tr_labels, te_images, te_labels, 60)

### Memory related information for imgs=90

In [ ]:
KNN_Information(1, 3, tr_images, tr_labels, te_images, te_labels, 90)

In [ ]:
KNN_Information(2, 3, tr_images, tr_labels, te_images, te_labels, 90)

In [ ]:
KNN_Information(3, 3, tr_images, tr_labels, te_images, te_labels, 90)

In [ ]:
KNN_Information(4, 3, tr_images, tr_labels, te_images, te_labels, 90)

In [ ]:
KNN_Information(5, 3, tr_images, tr_labels, te_images, te_labels, 90)

## With LSH

### Time and Accuracy

In [27]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,10,40,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008396148681640625 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 7.975128889083862 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008267879486083984 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 10.224925756454468 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008634567260742188 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 9.84305191040039 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008340120315551758 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 9.774850845336914 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008623123168945312 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         1
           9       0.67      1.00      0.80         2

    accuracy                           0.91        11
   macro avg       0.81      0.86      0.83        11
weighted avg       0.85      0.91      0.87        11

Time taken to evaluate KNN model on test portion: 8.169044971466064 seconds



In [28]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,10,40,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008124589920043945 seconds
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       1.00      1.00      1.00         2
           2       0.50      1.00      0.67         1
           3       0.00      0.00      0.00         0
           4       1.00      0.50      0.67         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       0.50      0.50      0.50         2

    accuracy                           0.73        11
   macro avg       0.75      0.69      0.69        11
weighted avg       0.86      0.73      0.76        11

Time taken to evaluate KNN model on test portion: 9.829199314117432 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.00875711441040039 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           8       0.00      0.00      0.00         0
           9       1.00      0.50      0.67         2

    accuracy                           0.91        11
   macro avg       0.88      0.81      0.83        11
weighted avg       1.00      0.91      0.94        11

Time taken to evaluate KNN model on test portion: 7.996341943740845 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008162260055541992 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       0.50      1.00      0.67         1
           9       1.00      0.50      0.67         2

    accuracy                           0.91        11
   macro avg       0.93      0.93      0.90        11
weighted avg       0.95      0.91      0.91        11

Time taken to evaluate KNN model on test portion: 9.698684215545654 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008324623107910156 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 8.731627941131592 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010239601135253906 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 8.563525676727295 seconds



In [29]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,10,60,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010272502899169922 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 9.918791055679321 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.01391744613647461 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 9.805070400238037 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.01120901107788086 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 8.420631408691406 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008884191513061523 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 9.859127283096313 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.012849092483520508 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 8.808597803115845 seconds



In [30]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,10,60,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011915206909179688 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         1
           9       0.67      1.00      0.80         2

    accuracy                           0.91        11
   macro avg       0.81      0.86      0.83        11
weighted avg       0.85      0.91      0.87        11

Time taken to evaluate KNN model on test portion: 8.28615117073059 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.0114593505859375 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       0.67      1.00      0.80         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      0.50      0.67         2

    accuracy                           0.91        11
   macro avg       0.95      0.93      0.92        11
weighted avg       0.94      0.91      0.90        11

Time taken to evaluate KNN model on test portion: 9.926749467849731 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.01199197769165039 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       0.50      1.00      0.67         1
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         1
           9       0.50      0.50      0.50         2

    accuracy                           0.64        11
   macro avg       0.50      0.56      0.52        11
weighted avg       0.59      0.64      0.61        11

Time taken to evaluate KNN model on test portion: 9.066958904266357 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011379480361938477 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 9.432212352752686 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011194944381713867 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Time taken to evaluate KNN model on test portion: 9.925613641738892 seconds



In [12]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,60,40,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008702993392944336 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      0.80      0.89         5
           3       0.80      0.80      0.80         5
           4       0.91      1.00      0.95        10
           5       0.83      0.71      0.77         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           8       0.00      0.00      0.00         0
           9       1.00      1.00      1.00         6

    accuracy                           0.93        61
   macro avg       0.85      0.83      0.84        61
weighted avg       0.95      0.93      0.94        61

Time taken to evaluate KNN model on test portion: 52.897632122039795 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009625911712646484 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.91      1.00      0.95        10
           2       1.00      0.80      0.89         5
           3       1.00      0.80      0.89         5
           4       1.00      0.90      0.95        10
           5       0.86      0.86      0.86         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           8       0.00      0.00      0.00         0
           9       0.86      1.00      0.92         6

    accuracy                           0.93        61
   macro avg       0.86      0.84      0.85        61
weighted avg       0.95      0.93      0.94        61

Time taken to evaluate KNN model on test portion: 50.967363119125366 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.007473945617675781 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.91      1.00      0.95        10
           2       1.00      0.80      0.89         5
           3       1.00      0.80      0.89         5
           4       0.90      0.90      0.90        10
           5       0.86      0.86      0.86         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           8       0.00      0.00      0.00         0
           9       1.00      1.00      1.00         6

    accuracy                           0.93        61
   macro avg       0.87      0.84      0.85        61
weighted avg       0.95      0.93      0.94        61

Time taken to evaluate KNN model on test portion: 52.69229316711426 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009661436080932617 seconds
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      1.00      1.00        10
           2       1.00      0.80      0.89         5
           3       1.00      1.00      1.00         5
           4       1.00      0.90      0.95        10
           5       1.00      1.00      1.00         7
           6       0.83      1.00      0.91         5
           7       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         6

    accuracy                           0.97        61
   macro avg       0.97      0.97      0.96        61
weighted avg       0.97      0.97      0.97        61

Time taken to evaluate KNN model on test portion: 51.13656735420227 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.007473945617675781 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      0.80      0.89         5
           4       0.83      1.00      0.91        10
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           8       0.00      0.00      0.00         0
           9       1.00      0.83      0.91         6

    accuracy                           0.95        61
   macro avg       0.88      0.85      0.86        61
weighted avg       0.97      0.95      0.96        61

Time taken to evaluate KNN model on test portion: 50.42148017883301 seconds



In [32]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,60,40,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010445117950439453 seconds
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      1.00      1.00        10
           2       0.83      1.00      0.91         5
           3       1.00      1.00      1.00         5
           4       1.00      0.70      0.82        10
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       0.75      1.00      0.86         6

    accuracy                           0.93        61
   macro avg       0.94      0.95      0.94        61
weighted avg       0.95      0.93      0.93        61

Time taken to evaluate KNN model on test portion: 50.85012769699097 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008497476577758789 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       0.71      1.00      0.83         5
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00        10
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00         5
           7       1.00      0.86      0.92         7
           9       1.00      1.00      1.00         6

    accuracy                           0.97        61
   macro avg       0.97      0.97      0.96        61
weighted avg       0.98      0.97      0.97        61

Time taken to evaluate KNN model on test portion: 51.537737131118774 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010043144226074219 seconds
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      0.80      0.89         5
           4       1.00      0.90      0.95        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       0.86      1.00      0.92         6

    accuracy                           0.97        61
   macro avg       0.97      0.97      0.96        61
weighted avg       0.97      0.97      0.97        61

Time taken to evaluate KNN model on test portion: 50.83741331100464 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008713722229003906 seconds
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       0.80      0.80      0.80         5
           4       0.90      0.90      0.90        10
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00         5
           7       0.88      1.00      0.93         7
           9       0.80      0.67      0.73         6

    accuracy                           0.92        61
   macro avg       0.91      0.91      0.91        61
weighted avg       0.92      0.92      0.92        61

Time taken to evaluate KNN model on test portion: 52.6326584815979 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.007035255432128906 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5
           4       1.00      0.90      0.95        10
           5       1.00      0.86      0.92         7
           6       0.83      1.00      0.91         5
           7       1.00      1.00      1.00         7
           9       0.86      1.00      0.92         6

    accuracy                           0.97        61
   macro avg       0.97      0.97      0.97        61
weighted avg       0.97      0.97      0.97        61

Time taken to evaluate KNN model on test portion: 52.27777075767517 seconds



In [33]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,60,60,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009248018264770508 seconds
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      0.60      0.75         5
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       0.86      1.00      0.92         6

    accuracy                           0.97        61
   macro avg       0.97      0.96      0.96        61
weighted avg       0.97      0.97      0.96        61

Time taken to evaluate KNN model on test portion: 51.120906352996826 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010677099227905273 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       0.83      1.00      0.91         5
           3       1.00      1.00      1.00         5
           4       1.00      0.90      0.95        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         6

    accuracy                           0.98        61
   macro avg       0.98      0.99      0.98        61
weighted avg       0.99      0.98      0.98        61

Time taken to evaluate KNN model on test portion: 53.33933115005493 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009388208389282227 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      0.80      0.89         5
           4       1.00      0.80      0.89        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           8       0.00      0.00      0.00         0
           9       0.75      1.00      0.86         6

    accuracy                           0.95        61
   macro avg       0.88      0.86      0.86        61
weighted avg       0.98      0.95      0.96        61

Time taken to evaluate KNN model on test portion: 51.89636421203613 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011330842971801758 seconds
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      0.80      0.89         5
           4       0.90      0.90      0.90        10
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           8       0.00      0.00      0.00         0
           9       1.00      1.00      1.00         6

    accuracy                           0.95        61
   macro avg       0.88      0.86      0.86        61
weighted avg       0.97      0.95      0.96        61

Time taken to evaluate KNN model on test portion: 51.08978748321533 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.011049985885620117 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       0.83      1.00      0.91         5
           4       1.00      0.90      0.95        10
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       0.86      1.00      0.92         6

    accuracy                           0.97        61
   macro avg       0.97      0.97      0.97        61
weighted avg       0.97      0.97      0.97        61

Time taken to evaluate KNN model on test portion: 51.81723666191101 seconds



In [34]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,60,60,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.013045787811279297 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         6

    accuracy                           1.00        61
   macro avg       1.00      1.00      1.00        61
weighted avg       1.00      1.00      1.00        61

Time taken to evaluate KNN model on test portion: 51.484586000442505 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010499954223632812 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.91      1.00      0.95        10
           2       1.00      1.00      1.00         5
           3       1.00      0.80      0.89         5
           4       1.00      0.90      0.95        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       0.86      1.00      0.92         6

    accuracy                           0.97        61
   macro avg       0.97      0.97      0.97        61
weighted avg       0.97      0.97      0.97        61

Time taken to evaluate KNN model on test portion: 51.775354862213135 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008810043334960938 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      0.80      0.89         5
           4       1.00      0.90      0.95        10
           5       0.88      1.00      0.93         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       0.86      1.00      0.92         6

    accuracy                           0.97        61
   macro avg       0.97      0.97      0.97        61
weighted avg       0.97      0.97      0.97        61

Time taken to evaluate KNN model on test portion: 52.31729173660278 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.013176441192626953 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      0.80      0.89         5
           3       0.80      0.80      0.80         5
           4       1.00      1.00      1.00        10
           5       0.88      1.00      0.93         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         6

    accuracy                           0.97        61
   macro avg       0.96      0.96      0.96        61
weighted avg       0.97      0.97      0.97        61

Time taken to evaluate KNN model on test portion: 49.94005537033081 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010164737701416016 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5
           4       1.00      0.90      0.95        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         7
           9       0.86      1.00      0.92         6

    accuracy                           0.98        61
   macro avg       0.98      0.99      0.99        61
weighted avg       0.99      0.98      0.98        61

Time taken to evaluate KNN model on test portion: 51.567237854003906 seconds



In [35]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,90,40,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008413076400756836 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       1.00      0.90      0.95        10
           4       1.00      0.77      0.87        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       0.88      1.00      0.93        14
           8       0.67      1.00      0.80         2
           9       0.80      0.89      0.84         9

    accuracy                           0.95        91
   macro avg       0.93      0.96      0.94        91
weighted avg       0.95      0.95      0.95        91

Time taken to evaluate KNN model on test portion: 75.60488104820251 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.01274871826171875 seconds
              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.92      1.00      0.96        12
           2       0.78      0.88      0.82         8
           3       1.00      0.90      0.95        10
           4       1.00      0.77      0.87        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       0.93      0.93      0.93        14
           8       1.00      0.50      0.67         2
           9       0.82      1.00      0.90         9

    accuracy                           0.92        91
   macro avg       0.93      0.90      0.90        91
weighted avg       0.93      0.92      0.92        91

Time taken to evaluate KNN model on test portion: 76.79811453819275 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008527040481567383 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       0.73      1.00      0.84         8
           3       1.00      0.90      0.95        10
           4       1.00      0.85      0.92        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       1.00      0.86      0.92        14
           8       1.00      0.50      0.67         2
           9       0.75      1.00      0.86         9

    accuracy                           0.93        91
   macro avg       0.95      0.91      0.92        91
weighted avg       0.95      0.93      0.94        91

Time taken to evaluate KNN model on test portion: 76.83764553070068 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008460521697998047 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       0.78      0.88      0.82         8
           3       0.90      0.90      0.90        10
           4       1.00      0.85      0.92        13
           5       1.00      0.71      0.83         7
           6       1.00      1.00      1.00         8
           7       0.93      1.00      0.97        14
           8       0.67      1.00      0.80         2
           9       0.80      0.89      0.84         9

    accuracy                           0.92        91
   macro avg       0.91      0.92      0.91        91
weighted avg       0.93      0.92      0.92        91

Time taken to evaluate KNN model on test portion: 75.00802397727966 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008342981338500977 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       0.93      0.93      0.93        14
           8       1.00      1.00      1.00         2
           9       0.89      0.89      0.89         9

    accuracy                           0.98        91
   macro avg       0.98      0.98      0.98        91
weighted avg       0.98      0.98      0.98        91

Time taken to evaluate KNN model on test portion: 77.11444306373596 seconds



In [36]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,90,40,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008326292037963867 seconds
              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       0.91      1.00      0.95        10
           4       0.92      0.85      0.88        13
           5       0.86      0.86      0.86         7
           6       0.88      0.88      0.88         8
           7       1.00      0.93      0.96        14
           8       1.00      0.50      0.67         2
           9       0.80      0.89      0.84         9

    accuracy                           0.92        91
   macro avg       0.92      0.89      0.90        91
weighted avg       0.93      0.92      0.92        91

Time taken to evaluate KNN model on test portion: 75.53918027877808 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.008266210556030273 seconds
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       0.89      0.80      0.84        10
           4       1.00      0.92      0.96        13
           5       0.86      0.86      0.86         7
           6       1.00      1.00      1.00         8
           7       0.88      1.00      0.93        14
           8       1.00      1.00      1.00         2
           9       1.00      0.78      0.88         9

    accuracy                           0.93        91
   macro avg       0.94      0.94      0.94        91
weighted avg       0.94      0.93      0.93        91

Time taken to evaluate KNN model on test portion: 77.4188826084137 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010015010833740234 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00        10
           4       0.91      0.77      0.83        13
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00         8
           7       0.93      1.00      0.97        14
           8       1.00      1.00      1.00         2
           9       0.73      0.89      0.80         9

    accuracy                           0.95        91
   macro avg       0.96      0.95      0.95        91
weighted avg       0.95      0.95      0.95        91

Time taken to evaluate KNN model on test portion: 75.4321939945221 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.0070078372955322266 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       0.89      1.00      0.94         8
           3       1.00      1.00      1.00        10
           4       0.90      0.69      0.78        13
           5       1.00      0.86      0.92         7
           6       0.89      1.00      0.94         8
           7       0.93      1.00      0.97        14
           8       1.00      0.50      0.67         2
           9       0.64      0.78      0.70         9

    accuracy                           0.91        91
   macro avg       0.92      0.88      0.89        91
weighted avg       0.92      0.91      0.91        91

Time taken to evaluate KNN model on test portion: 76.58163452148438 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009661436080932617 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       0.80      1.00      0.89         8
           3       1.00      0.80      0.89        10
           4       1.00      0.85      0.92        13
           5       1.00      1.00      1.00         7
           6       0.89      1.00      0.94         8
           7       0.93      0.93      0.93        14
           8       1.00      0.50      0.67         2
           9       0.73      0.89      0.80         9

    accuracy                           0.92        91
   macro avg       0.93      0.90      0.90        91
weighted avg       0.93      0.92      0.92        91

Time taken to evaluate KNN model on test portion: 75.72686862945557 seconds



In [37]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,90,60,0.8)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.00864410400390625 seconds
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       1.00      1.00      1.00        12
           2       0.80      1.00      0.89         8
           3       1.00      0.70      0.82        10
           4       1.00      0.85      0.92        13
           5       0.88      1.00      0.93         7
           6       1.00      1.00      1.00         8
           7       0.93      1.00      0.97        14
           8       1.00      0.50      0.67         2
           9       0.89      0.89      0.89         9

    accuracy                           0.92        91
   macro avg       0.93      0.89      0.90        91
weighted avg       0.93      0.92      0.92        91

Time taken to evaluate KNN model on test portion: 75.26719737052917 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010889530181884766 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       0.80      1.00      0.89         8
           3       0.90      0.90      0.90        10
           4       1.00      1.00      1.00        13
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00         8
           7       1.00      0.93      0.96        14
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         9

    accuracy                           0.97        91
   macro avg       0.97      0.97      0.97        91
weighted avg       0.97      0.97      0.97        91

Time taken to evaluate KNN model on test portion: 77.44342470169067 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.013187646865844727 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       0.89      1.00      0.94         8
           3       1.00      1.00      1.00        10
           4       1.00      0.85      0.92        13
           5       1.00      1.00      1.00         7
           6       0.89      1.00      0.94         8
           7       0.93      1.00      0.97        14
           8       1.00      0.50      0.67         2
           9       0.89      0.89      0.89         9

    accuracy                           0.96        91
   macro avg       0.96      0.92      0.93        91
weighted avg       0.96      0.96      0.95        91

Time taken to evaluate KNN model on test portion: 77.3489761352539 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009006500244140625 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       0.90      0.90      0.90        10
           4       1.00      1.00      1.00        13
           5       0.83      0.71      0.77         7
           6       0.89      1.00      0.94         8
           7       1.00      1.00      1.00        14
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         9

    accuracy                           0.97        91
   macro avg       0.96      0.96      0.96        91
weighted avg       0.97      0.97      0.97        91

Time taken to evaluate KNN model on test portion: 75.2674560546875 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010405302047729492 seconds
              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       0.92      1.00      0.96        12
           2       1.00      1.00      1.00         8
           3       1.00      0.90      0.95        10
           4       1.00      0.77      0.87        13
           5       1.00      1.00      1.00         7
           6       0.89      1.00      0.94         8
           7       0.93      0.93      0.93        14
           8       0.50      0.50      0.50         2
           9       0.70      0.78      0.74         9

    accuracy                           0.91        91
   macro avg       0.88      0.89      0.88        91
weighted avg       0.92      0.91      0.91        91

Time taken to evaluate KNN model on test portion: 78.44126391410828 seconds



In [38]:
LSH_KNN(1,3,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
LSH_KNN(2,3,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
LSH_KNN(3,3,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
LSH_KNN(4,3,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
LSH_KNN(5,3,tr_images,tr_labels,te_images,te_labels,90,60,0.9)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.013262033462524414 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       1.00      0.90      0.95        10
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        14
           8       0.67      1.00      0.80         2
           9       1.00      1.00      1.00         9

    accuracy                           0.99        91
   macro avg       0.97      0.99      0.97        91
weighted avg       0.99      0.99      0.99        91

Time taken to evaluate KNN model on test portion: 76.35568594932556 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010584354400634766 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       0.89      0.80      0.84        10
           4       0.86      0.92      0.89        13
           5       0.83      0.71      0.77         7
           6       1.00      1.00      1.00         8
           7       0.93      1.00      0.97        14
           8       0.50      1.00      0.67         2
           9       0.86      0.67      0.75         9

    accuracy                           0.91        91
   macro avg       0.89      0.91      0.89        91
weighted avg       0.92      0.91      0.91        91

Time taken to evaluate KNN model on test portion: 76.40368413925171 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.010660886764526367 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       0.89      1.00      0.94         8
           3       1.00      0.90      0.95        10
           4       1.00      0.85      0.92        13
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        14
           8       1.00      1.00      1.00         2
           9       0.82      1.00      0.90         9

    accuracy                           0.97        91
   macro avg       0.97      0.97      0.97        91
weighted avg       0.97      0.97      0.97        91

Time taken to evaluate KNN model on test portion: 77.39124321937561 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.009119510650634766 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       0.89      1.00      0.94         8
           3       1.00      0.90      0.95        10
           4       0.92      0.92      0.92        13
           5       1.00      0.86      0.92         7
           6       1.00      1.00      1.00         8
           7       1.00      0.93      0.96        14
           8       1.00      1.00      1.00         2
           9       0.82      1.00      0.90         9

    accuracy                           0.96        91
   macro avg       0.96      0.96      0.96        91
weighted avg       0.96      0.96      0.96        91

Time taken to evaluate KNN model on test portion: 78.17997074127197 seconds



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Time taken to fit KNN model: 0.012791872024536133 seconds
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00        10
           4       1.00      0.92      0.96        13
           5       1.00      1.00      1.00         7
           6       0.89      1.00      0.94         8
           7       1.00      0.93      0.96        14
           8       1.00      1.00      1.00         2
           9       0.90      1.00      0.95         9

    accuracy                           0.98        91
   macro avg       0.98      0.99      0.98        91
weighted avg       0.98      0.98      0.98        91

Time taken to evaluate KNN model on test portion: 76.60565996170044 seconds



### Memory related information for imgs=10

In [30]:
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
print(" ")
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
print(" ")
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
print(" ")
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9426.88 MiB
Peak memory usage during predict function: 9428.73 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9427.24 MiB
Virtual memory usage of Python process: 10598.03 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9427.24 MiB
Peak memory usage during predict function: 9428.75 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9427.50 MiB
Virtual memory usage of Python process: 10598.28 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9427.50 MiB
Peak memory usage during predict function: 9434.73 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9434.73 MiB
Virtual memory usage of Python process: 10605.51 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9434.73 MiB
Peak memory usage during predict function: 9435.75 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.00 MiB
Virtual memory usage of Python process: 10605.76 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [31]:
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
print(" ")
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
print(" ")
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
print(" ")
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.00 MiB
Peak memory usage during predict function: 9435.73 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9434.98 MiB
Virtual memory usage of Python process: 10605.76 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.01 MiB
Peak memory usage during predict function: 9435.73 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.23 MiB
Virtual memory usage of Python process: 10606.01 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.27 MiB
Peak memory usage during predict function: 9435.78 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.53 MiB
Virtual memory usage of Python process: 10606.26 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.53 MiB
Peak memory usage during predict function: 9435.75 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.80 MiB
Virtual memory usage of Python process: 10606.51 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [32]:
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
print(" ")
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
print(" ")
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
print(" ")
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.80 MiB
Peak memory usage during predict function: 9435.80 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.81 MiB
Virtual memory usage of Python process: 10606.51 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.84 MiB
Peak memory usage during predict function: 9435.95 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.95 MiB
Virtual memory usage of Python process: 10606.51 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.95 MiB
Peak memory usage during predict function: 9435.95 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.95 MiB
Virtual memory usage of Python process: 10606.51 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.98 MiB
Peak memory usage during predict function: 9435.98 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.98 MiB
Virtual memory usage of Python process: 10606.51 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [33]:
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
print(" ")
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
print(" ")
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
print(" ")
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.98 MiB
Peak memory usage during predict function: 9435.98 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.98 MiB
Virtual memory usage of Python process: 10606.51 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.98 MiB
Peak memory usage during predict function: 9435.98 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.98 MiB
Virtual memory usage of Python process: 10606.51 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.98 MiB
Peak memory usage during predict function: 9436.02 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9436.02 MiB
Virtual memory usage of Python process: 10606.51 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9436.02 MiB
Peak memory usage during predict function: 9436.02 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9436.02 MiB
Virtual memory usage of Python process: 10606.51 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 51
 


In [10]:
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,10,40,0.8)
print(" ")
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,10,40,0.9)
print(" ")
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,10,60,0.8)
print(" ")
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,10,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9427.71 MiB
Peak memory usage during predict function: 9435.80 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9435.81 MiB
Virtual memory usage of Python process: 10651.88 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9435.82 MiB
Peak memory usage during predict function: 9437.61 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9436.61 MiB
Virtual memory usage of Python process: 10652.61 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9445.81 MiB
Peak memory usage during predict function: 9447.45 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9447.45 MiB
Virtual memory usage of Python process: 10663.43 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9440.97 MiB
Peak memory usage during predict function: 9447.50 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9447.50 MiB
Virtual memory usage of Python process: 10663.43 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


### Memory related information for imgs=60

In [11]:
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
print(" ")
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
print(" ")
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
print(" ")
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9447.52 MiB
Peak memory usage during predict function: 9476.28 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9448.05 MiB
Virtual memory usage of Python process: 10663.93 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9448.06 MiB
Peak memory usage during predict function: 9466.21 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9438.40 MiB
Virtual memory usage of Python process: 10654.11 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9442.28 MiB
Peak memory usage during predict function: 9467.83 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9439.61 MiB
Virtual memory usage of Python process: 10655.27 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9442.15 MiB
Peak memory usage during predict function: 9466.99 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9438.88 MiB
Virtual memory usage of Python process: 10654.37 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


In [12]:
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
print(" ")
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
print(" ")
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
print(" ")
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9438.88 MiB
Peak memory usage during predict function: 9456.06 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9428.08 MiB
Virtual memory usage of Python process: 10643.57 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9430.27 MiB
Peak memory usage during predict function: 9469.97 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9428.11 MiB
Virtual memory usage of Python process: 10643.60 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9455.50 MiB
Peak memory usage during predict function: 9483.31 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9455.63 MiB
Virtual memory usage of Python process: 10671.07 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9455.63 MiB
Peak memory usage during predict function: 9483.41 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9455.63 MiB
Virtual memory usage of Python process: 10671.07 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


In [13]:
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
print(" ")
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
print(" ")
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
print(" ")
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9455.72 MiB
Peak memory usage during predict function: 9482.18 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9428.25 MiB
Virtual memory usage of Python process: 10643.60 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9430.48 MiB
Peak memory usage during predict function: 9457.00 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9428.78 MiB
Virtual memory usage of Python process: 10644.06 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9455.70 MiB
Peak memory usage during predict function: 9510.43 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9455.83 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9455.83 MiB
Peak memory usage during predict function: 9483.34 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9455.83 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


In [14]:
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
print(" ")
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
print(" ")
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
print(" ")
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9455.85 MiB
Peak memory usage during predict function: 9456.56 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9428.46 MiB
Virtual memory usage of Python process: 10643.85 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9430.62 MiB
Peak memory usage during predict function: 9456.57 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9428.91 MiB
Virtual memory usage of Python process: 10644.30 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9455.81 MiB
Peak memory usage during predict function: 9484.09 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9455.95 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9455.95 MiB
Peak memory usage during predict function: 9494.54 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.18 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


In [15]:
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,60,40,0.8)
print(" ")
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,60,40,0.9)
print(" ")
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,60,60,0.8)
print(" ")
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,60,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.18 MiB
Peak memory usage during predict function: 9483.01 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9428.71 MiB
Virtual memory usage of Python process: 10643.85 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9430.88 MiB
Peak memory usage during predict function: 9456.77 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9428.71 MiB
Virtual memory usage of Python process: 10643.85 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.09 MiB
Peak memory usage during predict function: 9483.87 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.21 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 56
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.21 MiB
Peak memory usage during predict function: 9483.89 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.14 MiB
Resident memory usage of Python process: 9456.35 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


### Memory related information for imgs=90

In [16]:
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
print(" ")
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
print(" ")
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
print(" ")
LSH_Information(1,3,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.40 MiB
Peak memory usage during predict function: 9497.91 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.40 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.40 MiB
Peak memory usage during predict function: 9497.89 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.40 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.40 MiB
Peak memory usage during predict function: 9539.59 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.44 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.44 MiB
Peak memory usage during predict function: 9536.93 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.44 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 53
 


In [17]:
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
print(" ")
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
print(" ")
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
print(" ")
LSH_Information(2,3,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.59 MiB
Peak memory usage during predict function: 9530.98 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.59 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.59 MiB
Peak memory usage during predict function: 9525.07 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.59 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.59 MiB
Peak memory usage during predict function: 9498.22 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.59 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.59 MiB
Peak memory usage during predict function: 9525.14 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.59 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


In [18]:
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
print(" ")
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
print(" ")
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
print(" ")
LSH_Information(3,3,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.61 MiB
Peak memory usage during predict function: 9498.04 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.61 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 
Peak memory usage during fit function: 9456.61 MiB
Peak memory usage during predict function: 9498.04 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.61 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.62 MiB
Peak memory usage during predict function: 9508.98 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.62 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 
Peak memory usage during fit function: 9456.62 MiB
Peak memory usage during predict function: 9508.98 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.62 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.63 MiB
Peak memory usage during predict function: 9498.13 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.63 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 
Peak memory usage during fit function: 9456.63 MiB
Peak memory usage during predict function: 9498.13 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.63 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9456.63 MiB
Peak memory usage during predict function: 9521.13 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.63 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 
Peak memory usage during fit function: 9456.63 MiB
Peak memory usage during predict function: 9521.13 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9456.63 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


In [9]:
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
print(" ")
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
print(" ")
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
print(" ")
LSH_Information(4,3,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9405.01 MiB
Peak memory usage during predict function: 9466.15 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9405.77 MiB
Virtual memory usage of Python process: 10572.30 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 55
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9405.56 MiB
Peak memory usage during predict function: 9455.99 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9405.88 MiB
Virtual memory usage of Python process: 10572.55 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 55
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9433.37 MiB
Peak memory usage during predict function: 9483.54 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9433.66 MiB
Virtual memory usage of Python process: 10600.27 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 55
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9418.73 MiB
Peak memory usage during predict function: 9490.61 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9418.75 MiB
Virtual memory usage of Python process: 10585.33 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 16
Number of open file descriptors of Python process: 55
 


In [20]:
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,90,40,0.8)
print(" ")
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,90,40,0.9)
print(" ")
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,90,60,0.8)
print(" ")
LSH_Information(5,3,tr_images,tr_labels,te_images,te_labels,90,60,0.9)
print(" ")

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.14 MiB
Peak memory usage during predict function: 9500.45 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9452.14 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.14 MiB
Peak memory usage during predict function: 9512.47 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9452.14 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.15 MiB
Peak memory usage during predict function: 9496.46 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9452.15 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Peak memory usage during fit function: 9452.15 MiB
Peak memory usage during predict function: 9493.54 MiB
L1 cache size: 32.00 KiB
L2 cache size: 256.00 KiB
Memory usage of KNN object and its attributes: 0.00 MiB
Resident memory usage of Python process: 9452.15 MiB
Virtual memory usage of Python process: 10671.32 MiB
CPU usage of Python process: 0.00%
Number of threads of Python process: 17
Number of open file descriptors of Python process: 61
 
